# Исследование надежности заемщиков


Заказчик — кредитный отдел банка.

**Задача:**
Выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте таблицу и изучите общую информацию о данных

**1. Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраните в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

**15. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.
Создадим функцию с именем `categorize_income()` и применим ее.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Сгруппируем по количеству детей. Рассчитаем количество невозвратных кредитов, которое соответствует сумме в колонке 'debt' и общее количество кредитов в каждой категориии.
На основании полученных выше данных рассчитаем процент невозвратных кредитов для каждой категории.

In [22]:
children_debt = data.groupby('children').agg({'debt': ['sum', 'count']})
children_debt['non_return'] = children_debt['debt']['sum'] / children_debt['debt']['count'] * 100
display(children_debt) # Ваш код будет здесь. Вы можете создавать новые ячейки.

debt        non_return
           sum  count           
children                        
0         1063  14107   7.535266
1          444   4809   9.232689
2          194   2052   9.454191
3           27    330   8.181818
4            4     41   9.756098
5            0      9   0.000000

**Вывод:** 
В категории с 5 детьми количество кредитов - 9, что составляет всего 0,04% от общего количества взятых кредитов. Поэтому данная группа не является показательной.
В категории с 4 детьми количество кредитов - 41, что составляет 0,19% от общего количества кредитов и не является показательной.
В категории без детей процент невозвращенных в срок кредитов самый низкий.
В категориях с 1 и 2 детьми процент не возврата кредита в срок примерно одинаковый и выше примерно на 2%, чем в группе без детей. Насколько это существенно необходимо уточнить у специалистов кредитного отдела банка.

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Сгруппируем по семейному положению и рассчитаем процент невозвратных кредитов для каждой группы, как отношение колличества невозвращенных в срок кредитов к количеству кредитов в каждой группе.

In [23]:
family_status_debt = data.groupby('family_status').agg({'debt': ['sum', 'count']})
family_status_debt['non_return'] = family_status_debt['debt']['sum'] / family_status_debt['debt']['count'] * 100
display(family_status_debt.sort_values(by='non_return')) # Ваш код будет здесь. Вы можете создавать новые ячейки.

debt        non_return
                       sum  count           
family_status                               
вдовец / вдова          63    951   6.624606
в разводе               84   1189   7.064760
женат / замужем        927  12266   7.557476
гражданский брак       385   4146   9.286059
Не женат / не замужем  273   2796   9.763948

**Вывод:** 
Для групп Не женат/не замужем и гражданский брак процент невозврата кредитов в срок самый высокий.

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Сгруппируем на основании категорий по уровню дохода 'total_income_category' и рассчитаем долю невозвратных кредитов в процентах.

In [24]:
total_income_debt = data.groupby('total_income_category').agg({'debt': ['sum', 'count']})
total_income_debt['non_return'] = total_income_debt['debt']['sum'] / total_income_debt['debt']['count'] * 100
display(total_income_debt)

debt        non_return
                        sum  count           
total_income_category                        
A                         2     25   8.000000
B                       354   5014   7.060231
C                      1353  15938   8.489145
D                        21    349   6.017192
E                         2     22   9.090909

**Вывод:** 
- Для категории с доходом 1000001 и выше - 'A' количество кредитов составляет 0,11% от общего количества кредитов и не является показательным
- Для категории 200001-1000000 - 'B' процент невозвращенных в срок кредитов ниже, чем в категория 'C'
- В группе с доходом 50001-200000 - 'C' процент невозвращенных в срок кредитов является наиболее высоким, но при этом это самая большая группа по количеству кредитов и составляет 74,66%
- В группе с доходом 30000-50000 - 'D' процент не возврата кредитов в срок минимальный из тех групп, которые являются показательными. Но данная группа составляет 1,6% от общего числа кредитов
- Категория с доходом 0-30000 - 'E' составляет 0,1% от общего количества кредитов и не является показательной


**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

Сгруппируем по категориям на основании цели кредита 'purpose_category' и рассчитаем долю невозвратных кредитов в процентах.

In [25]:
purpose_debt = data.groupby('purpose_category').agg({'debt': ['sum', 'count']})
purpose_debt['non_return'] = purpose_debt['debt']['sum'] / purpose_debt['debt']['count'] * 100
print(purpose_debt.sort_values(by='non_return')) # Ваш код будет здесь. Вы можете создавать новые ячейки.

                         debt        non_return
                          sum  count           
purpose_category                               
операции с недвижимостью  780  10754   7.253115
проведение свадьбы        183   2324   7.874355
получение образования     369   3989   9.250439
операции с автомобилем    400   4281   9.343611


**Вывод:** 
Процент невозвратных в срок кредитов для категорий с целью: "получение образования" и "операции с автомобилем" выше, чем для категорий: "операции с недвижимостью" и "проведение свадьбы"

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* 
Данные могут быть пропущены по ряду причин:
• человеческий фактор - данные были пропущены при внесении сотрудником
• специально не указанные данные
• технические ошибки при выгрузке или переносе данных

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* 
Использование медианных значений позволяет избежать ошибок, если какие-то данные сильно отличаются от большинства. Такой принцип замены переменных применим для зарплаты/уровня дохода. Но важно, чтобы этот показатель не был намеренно скрыт, как например, в случае если доход слишком низкий или совсем отсутствует. Чтобы этого избежать использовали замену пропущенного дохода по категориям в зависимости от типа занятости.

## Общий вывод.

Наибольшие риски будет представлять выдача кредита людям, которые по всем параметрам имеют более высокий процент невозвратных в срок кредитов. В нашем случае это будут люди со следующими категориями:
- не женатые/не замужем или в гражданском браке
- с одним или двумя детьми
- с доходом от 50001 до 200000 рублей
- если кредит берется для получения образования или операций с автомобилем
Наибольший процент возврата кредитов в срок будет для групп, объединяющих параметры с минимальной долей невозвратных кредитов:
- женат/замужем или в разводе или вдова/вдовец
- без детей
- с доходом от 200001 до 10000000
